In [66]:
import spacy
import numpy as np
import pandas as pd

In [67]:
nlp = spacy.load('en_core_web_sm')
# nlp = spacy.load('fr_core_news_sm')

In [68]:
def treat_comment(comment):
    spacy_comment = nlp(comment, disable=["parser", "tagger", "ner", "textcat"])
    treated_tokens = [w.text for w in spacy_comment if w.is_alpha and not w.is_stop]
    return " ".join(treated_tokens)

In [69]:
mainCSV = pd.read_csv('./data/train_fr.csv', index_col="id")

In [70]:
mainCSV.shape

(159571, 7)

In [71]:
mainCSV.columns

Index(['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [72]:
mainCSV.duplicated().sum()

146

In [73]:
df = mainCSV.drop_duplicates()

In [74]:

Y = df[['toxic']]

In [75]:
from sklearn.linear_model import LogisticRegression

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()


X = vectorizer.fit_transform(df['comment_text'])

import pickle
# save the model to disk
# Sauvegardez le vectorizer dans un fichier
with open('toxic_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)


In [77]:
X

<159425x166249 sparse matrix of type '<class 'numpy.float64'>'
	with 6794976 stored elements in Compressed Sparse Row format>

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [79]:
model = LogisticRegression()
model.fit(X_train, Y_train)

c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [80]:
model.score(X_train, Y_train)

0.9568780944734377

In [81]:
import pickle

In [82]:
# save the model to disk
filename = 'toxic_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [83]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [84]:
y_pred = model.predict(X_test)

In [85]:
accuracy_score(Y_test, y_pred)

0.9518277843289761

In [86]:
confusion_matrix(Y_test, y_pred)

array([[35843,   219],
       [ 1701,  2094]], dtype=int64)

In [87]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [88]:
model2 = Sequential()
model2.add(Embedding(2000, 128, input_length = X.shape[-1]))
model2.add(SpatialDropout1D(0.4))
model2.add(LSTM(60, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(2,activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 166249, 128)       256000    
                                                                 
 spatial_dropout1d_2 (Spati  (None, 166249, 128)       0         
 alDropout1D)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 60)                45360     
                                                                 
 dense_2 (Dense)             (None, 2)                 122       
                                                                 
Total params: 301482 (1.15 MB)
Trainable params: 301482 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [89]:
df['toxic']

id
0000997932d777bf    0
000103f0d9cfb60f    0
000113f07ec002fd    0
0001b41b1c6bb37e    0
0001d958c54c6e35    0
                   ..
ffe987279560d7ff    0
ffea4adeee384e90    0
ffee36eab5c267c9    0
fff125370e4aaaf3    0
fff46fc426af1f9a    0
Name: toxic, Length: 159425, dtype: int64

In [90]:
data = pd.DataFrame({'text':df['comment_text'], 'sentiment':df['toxic']})
data = data.replace({1.0:'positive',2.0:'negative'})
data.head()

,text,sentiment
id,,
0000997932d777bf,Explanation\nWhy have the changes made under m...,0
000103f0d9cfb60f,D'aww! It corresponds to that background color...,0
000113f07ec002fd,"Hey man, I'm not really trying to edit the war...",0
0001b41b1c6bb37e,"""\nMore\nI can not make any real suggestions f...",0
0001d958c54c6e35,"You, sir, are my hero. Is there a chance that ...",0


In [91]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['text'].values)

In [92]:
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [93]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(119568, 3076) (119568, 2)
(39857, 3076) (39857, 2)


In [94]:
batch_size = 32
model2.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7


ValueError: in user code:

    File "c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\basti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   